In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
T = 6 # length of observation sequence
N = 2 # number of states in model
M = 3 # number of observations model
V = {1, 2, 3} # set of possible observations

# States

In [3]:
# create space and initial state probabilities
states = ['rainy', 'sunny']
pi = [0.6, 0.4]

state_space = pd.Series(pi, index=states, name='states')
print(state_space)
print(state_space.sum())

rainy    0.6
sunny    0.4
Name: states, dtype: float64
1.0


# Transition Matrix

In [4]:
# create transition matrix

a_df = pd.DataFrame(columns=states, index=states)
a_df.loc[states[0]] = [0.7, 0.3]
a_df.loc[states[1]] = [0.4, 0.6]

print(a_df)

A = a_df.values
print('\n', A, A.shape, '\n')
print(a_df.sum(axis=1))

      rainy sunny
rainy   0.7   0.3
sunny   0.4   0.6

 [[0.7 0.3]
 [0.4 0.6]] (2, 2) 

rainy    1.0
sunny    1.0
dtype: float64


# Emision Matrix

In [5]:
emision = ['walk', 'shop', 'clean']
b_df = pd.DataFrame(columns=emision, index=states)
b_df.loc[states[0]] = [0.1, 0.4, 0.5]
b_df.loc[states[1]] = [0.6, 0.3, 0.1]

print(b_df)

B = b_df.values
print('\n', B, B.shape, '\n')
print(b_df.sum(axis=1))

      walk shop clean
rainy  0.1  0.4   0.5
sunny  0.6  0.3   0.1

 [[0.1 0.4 0.5]
 [0.6 0.3 0.1]] (2, 3) 

rainy    1.0
sunny    1.0
dtype: float64


# 1. Computing Likelihoods

In [6]:
O = [2, 2, 3, 1, 3, 2]

In [7]:
# initialization
column_alpha = [i for i in range(N)]
index_alpha = [i for i in range(T)]
alpha = pd.DataFrame(np.nan, index=index_alpha, columns=column_alpha)
alpha = alpha.values

for i in range(N):
    alpha[0,i] = pi[i] * B[i, O[0]-1]

print(alpha)

[[0.24 0.12]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]]


In [8]:
# induction
for t in range(T-1):
    for j in range(N):
        u = 0
        for i in range(N):
            u = u + alpha[t,i]*A[i,j]
        alpha[t+1, j] = u*B[j, O[t+1]-1]

print(alpha)

[[0.24       0.12      ]
 [0.0864     0.0432    ]
 [0.03888    0.005184  ]
 [0.00292896 0.00886464]
 [0.00279806 0.00061975]
 [0.00088262 0.00036338]]


In [9]:
# evaluating the probability
prob = 0
for i in range(N):
    prob = prob+alpha[T-1,i]

print(prob)

0.001245997728


# 2. Decoding Hidden States

## a. Dynamic Programming Solution

In [10]:
p = np.zeros((2,)*6, dtype=np.float64)

In [11]:
for i in range(N):
    for j in range(N):
        for k in range(N):
            for l in range(N):
                for m in range(N):
                    for n in range(N):
                        p[i,j,k,l,m,n] = pi[i]*B[i,O[0]-1]* \
                                         A[i,j]*B[j,O[1]-1]* \
                                         A[j,k]*B[k,O[2]-1]* \
                                         A[k,l]*B[l,O[3]-1]* \
                                         A[l,m]*B[m,O[4]-1]* \
                                         A[m,n]*B[n,O[5]-1]

In [12]:
print(p[:,:,1,1,1,1])

[[7.838208e-06 5.038848e-06]
 [2.239488e-06 5.038848e-06]]


In [13]:
sumprob = 0
for i in range(N):
    for j in range(N):
        for k in range(N):
            for l in range(N):
                for m in range(N):
                    for n in range(N):
                        sumprob = sumprob + p[i,j,k,l,m,n]

In [14]:
print(sumprob)

0.001245997728


In [15]:
pnorm = np.zeros((2,)*6, dtype=np.float64)

In [16]:
for i in range(N):
    for j in range(N):
        for k in range(N):
            for l in range(N):
                for m in range(N):
                    for n in range(N):
                        pnorm[i,j,k,l,m,n] = p[i,j,k,l,m,n]/sumprob

In [17]:
print(pnorm[:,:,1,1,1,1])

[[0.00629071 0.00404403]
 [0.00179735 0.00404403]]


In [18]:
[C, I] = np.max(pnorm.flatten(order='F')), np.argmax(pnorm.flatten(order='F'))
print([C, I])

[0.19027450425655987, 8]


In [19]:
[I1, I2, I3, I4, I5, I6] = np.unravel_index(I, pnorm.shape, 'F')
DP_index = [I1, I2, I3, I4, I5, I6]
print(DP_index)

[0, 0, 0, 1, 0, 0]


In [20]:
DP_maximal_probability = pnorm[I1, I2, I3, I4, I5, I6]*sumprob
print(DP_maximal_probability)

0.0002370815999999999


In [21]:
DP_normalized_maximal_probability = pnorm[I1, I2, I3, I4, I5, I6]
print(DP_normalized_maximal_probability)

0.19027450425655987


## b. HMM Solution

In [22]:
# initialization
beta = np.zeros((T, N))
for i in range(N):
    beta[T-1, i] = 1
print(beta)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]]


In [23]:
# induction
for t in range(T-2, -1, -1):
    for i in range(N):
        for j in range(N):
            beta[t, i] = beta[t,i]+A[i,j]*B[j, O[t+1]-1]*beta[t+1,j]
print(beta)

[[0.00365153 0.00308026]
 [0.01055701 0.00772852]
 [0.026771   0.039572  ]
 [0.1397     0.0944    ]
 [0.37       0.34      ]
 [1.         1.        ]]


In [24]:
# gamma
gamma = np.zeros((T, N), dtype=float)
for t in range(T):
    for i in range(N):
        gamma[t, i] = alpha[t, i]*beta[t, i]/prob
print(gamma)

[[0.70334567 0.29665433]
 [0.7320444  0.2679556 ]
 [0.83535985 0.16464015]
 [0.32839202 0.67160798]
 [0.83088729 0.16911271]
 [0.70836202 0.29163798]]


In [25]:
result = []
for i in gamma:
    result.append(np.argmax(i))
print(result)

[0, 0, 0, 1, 0, 0]


In [27]:
di_gamma = np.zeros((T-1, N, N))
for t in range(T-1):
    for i in range(N):
        for j in range(N):
            di_gamma[t,i,j] = (alpha[t,i]*A[i,j]*B[j,O[t+1]-1]*beta[t+1,j])/prob
print(di_gamma[:,:,1])

[[0.1339778  0.1339778 ]
 [0.08232007 0.08232007]
 [0.53021683 0.14139115]
 [0.02397708 0.14513562]
 [0.20210772 0.08953026]]
